In [1]:
# Imports we may need
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import ujson as json
import bz2
import pyarrow.parquet as pq

from wikidata.client import Client
import wikipedia

import urllib.request

client = Client()

In [5]:
#read YYYY quotes file

#chunks_2008 = pd.read_json("data/quotes-2008.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
#chunks_2009 = pd.read_json("data/quotes-2009.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
#chunks_2010 = pd.read_json("data/quotes-2010.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
#chunks_2011 = pd.read_json("data/quotes-2011.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
#chunks_2012 = pd.read_json("data/quotes-2012.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
#chunks_2013 = pd.read_json("data/quotes-2013.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
#chunks_2014 = pd.read_json("data/quotes-2014.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
chunks_2015 = pd.read_json("data/quotes-2015.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
chunks_2016 = pd.read_json("data/quotes-2016.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
chunks_2017 = pd.read_json("data/quotes-2017.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
chunks_2018 = pd.read_json("data/quotes-2018.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
chunks_2019 = pd.read_json("data/quotes-2019.json.bz2",lines=True, chunksize = 10000, compression = "bz2")
chunks_2020 = pd.read_json("data/quotes-2020.json.bz2",lines=True, chunksize = 10000, compression = 'bz2')

chunks_YYYY = [chunks_2015, chunks_2016, chunks_2017, chunks_2018, chunks_2019, chunks_2020]

years_list= ["2015", "2016", "2017", "2018","2019", "2020"]

In [12]:
def extract_dataframe(word, year):
    list_df = []
    with pd.read_json("data/quotes-"+year+".json.bz2", lines=True,  chunksize = 10000, compression = 'bz2') as df_reader:
        for chunk in df_reader:
            chunk_word = chunk[chunk['quotation'].str.contains(word)]
            list_df.append(chunk_word)
    df_word = pd.concat(list_df)
    #we keep just the author with the highest probability
    df_word['probas'] =df_word['probas'].transform( lambda x : x[0])
    
    #we delete the rows with a probability lower than 0.7 and with a quote attributed to None
    rows_to_delete = []
    for i in range(df_word['probas'].values.shape[0]):
        if df_word['probas'].values[i][0] == 'None' or float(df_word['probas'].values[i][1])<0.7:
            rows_to_delete.append(df_word.index[i])
    df_word.drop(rows_to_delete)
    return df_word

In [14]:
for year in years_list:
    list_df_abortion = []
    one_df_abortion = extract_dataframe("abortion", year)
    print('i finish')
    list_df_abortion.append(one_df_abortion)
df_abortion = pd.concat(list_df_abortion)

i finish
i finish
i finish
i finish
i finish
i finish


In [15]:
df_abortion

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
331,2020-03-03-010183,cultural movement centered on a set of social ...,Katherine Stewart,"[Q21466308, Q43390345, Q43392594, Q6376550]",2020-03-03 00:00:00,2,"[Katherine Stewart, 0.6689]",[https://www.rawstory.com/2020/03/hot-topics-i...,E
2132,2020-04-02-073192,We are pleased that the court recognized the u...,Marjorie Dannenfelser,[Q6766244],2020-04-02 16:51:00,2,"[Marjorie Dannenfelser, 0.8069]",[https://www.catholicnews.com/services/english...,E
3022,2020-01-24-022010,has knowledge that the pregnant woman is seeki...,John Kasich,[Q69319],2020-01-24 20:52:00,2,"[John Kasich, 0.5464]",[http://lifesitenews.com/news/trump-admin-back...,E
7143,2020-03-09-067131,view abortion as a choice and a right. I think...,Joe Biden,[Q6279],2020-03-09 13:00:23,1,"[Joe Biden, 0.8478]",[http://nymag.com/intelligencer/2020/03/what-d...,E
7278,2020-03-06-075622,We know that it's all going to come down to th...,None,[],2020-03-06 20:50:34,1,"[None, 0.7558]",[http://www.christianpost.com/weekly-briefing/...,E
...,...,...,...,...,...,...,...,...,...
5241877,2020-03-02-001712,"abortion isn't a human right, it takes a human...",Mike Pompeo,[Q473239],2020-03-02 21:00:00,1,"[Mike Pompeo, 0.5879]",[https://theslot.jezebel.com/a-random-list-of-...,E
5241924,2020-03-04-003112,"among medical procedures, first trimester abor...",Ruth Bader Ginsburg,[Q11116],2020-03-04 14:43:00,15,"[Ruth Bader Ginsburg, 0.8717]",[http://uspolitics.einnews.com/article/5111889...,E
5242770,2020-01-24-042870,"In California, we will continue to protect our...",Xavier Becerra,[Q1855840],2020-01-24 19:08:19,9,"[Xavier Becerra, 0.8155]",[http://www.herald-dispatch.com/ap/ap_nation/c...,E
5243526,2020-03-09-051639,that would have set a standard of care for chi...,None,[],2020-03-09 19:31:00,1,"[None, 0.6997]",[https://www.lifenews.com/2020/03/09/tucker-ca...,E


In [ ]:
list_abortion = []
#year = '2020'
#for year in years_list:
with pd.read_json("data/quotes-2020.json.bz2",lines=True, chunksize = 10000, compression = 'bz2') as df_reader:
    for chunk in df_reader:
        chunk_abortion = chunk[chunk['quotation'].str.contains("abortion")]
        list_abortion.append(chunk_abortion)
df_abortion = pd.concat(list_abortion)

In [6]:
df_abortion['probas'] =df_abortion['probas'].transform( lambda x : x[0])

In [7]:
df_abortion

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
331,2020-03-03-010183,cultural movement centered on a set of social ...,Katherine Stewart,"[Q21466308, Q43390345, Q43392594, Q6376550]",2020-03-03 00:00:00,2,"[Katherine Stewart, 0.6689]",[https://www.rawstory.com/2020/03/hot-topics-i...,E
2132,2020-04-02-073192,We are pleased that the court recognized the u...,Marjorie Dannenfelser,[Q6766244],2020-04-02 16:51:00,2,"[Marjorie Dannenfelser, 0.8069]",[https://www.catholicnews.com/services/english...,E
3022,2020-01-24-022010,has knowledge that the pregnant woman is seeki...,John Kasich,[Q69319],2020-01-24 20:52:00,2,"[John Kasich, 0.5464]",[http://lifesitenews.com/news/trump-admin-back...,E
7143,2020-03-09-067131,view abortion as a choice and a right. I think...,Joe Biden,[Q6279],2020-03-09 13:00:23,1,"[Joe Biden, 0.8478]",[http://nymag.com/intelligencer/2020/03/what-d...,E
7278,2020-03-06-075622,We know that it's all going to come down to th...,None,[],2020-03-06 20:50:34,1,"[None, 0.7558]",[http://www.christianpost.com/weekly-briefing/...,E
...,...,...,...,...,...,...,...,...,...
5241877,2020-03-02-001712,"abortion isn't a human right, it takes a human...",Mike Pompeo,[Q473239],2020-03-02 21:00:00,1,"[Mike Pompeo, 0.5879]",[https://theslot.jezebel.com/a-random-list-of-...,E
5241924,2020-03-04-003112,"among medical procedures, first trimester abor...",Ruth Bader Ginsburg,[Q11116],2020-03-04 14:43:00,15,"[Ruth Bader Ginsburg, 0.8717]",[http://uspolitics.einnews.com/article/5111889...,E
5242770,2020-01-24-042870,"In California, we will continue to protect our...",Xavier Becerra,[Q1855840],2020-01-24 19:08:19,9,"[Xavier Becerra, 0.8155]",[http://www.herald-dispatch.com/ap/ap_nation/c...,E
5243526,2020-03-09-051639,that would have set a standard of care for chi...,None,[],2020-03-09 19:31:00,1,"[None, 0.6997]",[https://www.lifenews.com/2020/03/09/tucker-ca...,E


In [16]:
# we save it (checkpoint)
df_abortion.to_csv('df_abortion.csv', index=False)

In [17]:
df_abortion_copy = pd.read_csv('df_abortion.csv')

In [18]:
df_abortion_copy

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2020-03-03-010183,cultural movement centered on a set of social ...,Katherine Stewart,"['Q21466308', 'Q43390345', 'Q43392594', 'Q6376...",2020-03-03 00:00:00,2,"['Katherine Stewart', '0.6689']",['https://www.rawstory.com/2020/03/hot-topics-...,E
1,2020-04-02-073192,We are pleased that the court recognized the u...,Marjorie Dannenfelser,['Q6766244'],2020-04-02 16:51:00,2,"['Marjorie Dannenfelser', '0.8069']",['https://www.catholicnews.com/services/englis...,E
2,2020-01-24-022010,has knowledge that the pregnant woman is seeki...,John Kasich,['Q69319'],2020-01-24 20:52:00,2,"['John Kasich', '0.5464']",['http://lifesitenews.com/news/trump-admin-bac...,E
3,2020-03-09-067131,view abortion as a choice and a right. I think...,Joe Biden,['Q6279'],2020-03-09 13:00:23,1,"['Joe Biden', '0.8478']",['http://nymag.com/intelligencer/2020/03/what-...,E
4,2020-03-06-075622,We know that it's all going to come down to th...,None,[],2020-03-06 20:50:34,1,"['None', '0.7558']",['http://www.christianpost.com/weekly-briefing...,E
...,...,...,...,...,...,...,...,...,...
4056,2020-03-02-001712,"abortion isn't a human right, it takes a human...",Mike Pompeo,['Q473239'],2020-03-02 21:00:00,1,"['Mike Pompeo', '0.5879']",['https://theslot.jezebel.com/a-random-list-of...,E
4057,2020-03-04-003112,"among medical procedures, first trimester abor...",Ruth Bader Ginsburg,['Q11116'],2020-03-04 14:43:00,15,"['Ruth Bader Ginsburg', '0.8717']",['http://uspolitics.einnews.com/article/511188...,E
4058,2020-01-24-042870,"In California, we will continue to protect our...",Xavier Becerra,['Q1855840'],2020-01-24 19:08:19,9,"['Xavier Becerra', '0.8155']",['http://www.herald-dispatch.com/ap/ap_nation/...,E
4059,2020-03-09-051639,that would have set a standard of care for chi...,None,[],2020-03-09 19:31:00,1,"['None', '0.6997']",['https://www.lifenews.com/2020/03/09/tucker-c...,E


In [8]:
#df_abortion.reset_index()
#df_abortion.drop(columns = 'index')
df_abortion

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
331,2020-03-03-010183,cultural movement centered on a set of social ...,Katherine Stewart,"[Q21466308, Q43390345, Q43392594, Q6376550]",2020-03-03 00:00:00,2,"[Katherine Stewart, 0.6689]",[https://www.rawstory.com/2020/03/hot-topics-i...,E
2132,2020-04-02-073192,We are pleased that the court recognized the u...,Marjorie Dannenfelser,[Q6766244],2020-04-02 16:51:00,2,"[Marjorie Dannenfelser, 0.8069]",[https://www.catholicnews.com/services/english...,E
3022,2020-01-24-022010,has knowledge that the pregnant woman is seeki...,John Kasich,[Q69319],2020-01-24 20:52:00,2,"[John Kasich, 0.5464]",[http://lifesitenews.com/news/trump-admin-back...,E
7143,2020-03-09-067131,view abortion as a choice and a right. I think...,Joe Biden,[Q6279],2020-03-09 13:00:23,1,"[Joe Biden, 0.8478]",[http://nymag.com/intelligencer/2020/03/what-d...,E
7278,2020-03-06-075622,We know that it's all going to come down to th...,None,[],2020-03-06 20:50:34,1,"[None, 0.7558]",[http://www.christianpost.com/weekly-briefing/...,E
...,...,...,...,...,...,...,...,...,...
5241877,2020-03-02-001712,"abortion isn't a human right, it takes a human...",Mike Pompeo,[Q473239],2020-03-02 21:00:00,1,"[Mike Pompeo, 0.5879]",[https://theslot.jezebel.com/a-random-list-of-...,E
5241924,2020-03-04-003112,"among medical procedures, first trimester abor...",Ruth Bader Ginsburg,[Q11116],2020-03-04 14:43:00,15,"[Ruth Bader Ginsburg, 0.8717]",[http://uspolitics.einnews.com/article/5111889...,E
5242770,2020-01-24-042870,"In California, we will continue to protect our...",Xavier Becerra,[Q1855840],2020-01-24 19:08:19,9,"[Xavier Becerra, 0.8155]",[http://www.herald-dispatch.com/ap/ap_nation/c...,E
5243526,2020-03-09-051639,that would have set a standard of care for chi...,None,[],2020-03-09 19:31:00,1,"[None, 0.6997]",[https://www.lifenews.com/2020/03/09/tucker-ca...,E


In [9]:
rows_to_delete = []
for i in range(df_abortion['probas'].values.shape[0]):
    if df_abortion['probas'].values[i][0] == 'None' or float(df_abortion['probas'].values[i][1])<0.7:
        rows_to_delete.append(df_abortion.index[i])
df_abortion.drop(rows_to_delete)

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
2132,2020-04-02-073192,We are pleased that the court recognized the u...,Marjorie Dannenfelser,[Q6766244],2020-04-02 16:51:00,2,"[Marjorie Dannenfelser, 0.8069]",[https://www.catholicnews.com/services/english...,E
7143,2020-03-09-067131,view abortion as a choice and a right. I think...,Joe Biden,[Q6279],2020-03-09 13:00:23,1,"[Joe Biden, 0.8478]",[http://nymag.com/intelligencer/2020/03/what-d...,E
10926,2020-01-28-038080,I would hope it makes a person choosing to hav...,Curt Bramble,[Q5195354],2020-01-28 00:37:23,4,"[Curt Bramble, 0.7074]",[http://utahpolicy.com/index.php/features/toda...,E
16783,2020-01-23-081112,The Constitution of the State of Iowa does not...,Gov. Kim Reynolds,[Q6409269],2020-01-23 12:00:00,1,"[Gov. Kim Reynolds, 0.8518]",[http://thegazette.com/subject/opinion/staff-c...,E
30087,2020-02-17-074727,Walker is making the following claim: If you t...,Michael Gerson,[Q6830587],2020-02-17 19:37:33,4,"[Michael Gerson, 0.7799]",[https://www.nationalreview.com/corner/no-pro-...,E
...,...,...,...,...,...,...,...,...,...
5238384,2020-03-20-060725,The fact that these abortion facilities have a...,Molly Smith,"[Q22963284, Q6896679]",2020-03-20 22:17:55,1,"[Molly Smith, 0.756]",[https://www.lifenews.com/2020/03/20/abortion-...,E
5239587,2020-01-20-011325,deeply entangled with the abortion lobby.,Lila Rose,[Q6547490],2020-01-20 16:56:20,3,"[Lila Rose, 0.793]",[https://dailycaller.com/2020/01/20/abortion-s...,E
5240741,2020-02-14-070984,reduce maternal mortality and morbidity arisin...,Smriti Irani,[Q7546539],2020-02-14 13:30:18,1,"[Smriti Irani, 0.7545]",[https://www.nationalrighttolifenews.org/2020/...,E
5241924,2020-03-04-003112,"among medical procedures, first trimester abor...",Ruth Bader Ginsburg,[Q11116],2020-03-04 14:43:00,15,"[Ruth Bader Ginsburg, 0.8717]",[http://uspolitics.einnews.com/article/5111889...,E
